In [1]:
pip install transformers pythainlp torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install fuzzywuzzy[speedup]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

In [4]:
from tqdm.notebook import tqdm
import time

# ตัวอย่าง Progress Bar
for i in tqdm(range(10)):
    time.sleep(0.5)

  0%|          | 0/10 [00:00<?, ?it/s]

In [5]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# โหลด tokenizer และโมเดล WangchanBERTa ที่ปรับใช้กับภาษาไทย
tokenizer = AutoTokenizer.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased", use_fast=False)
model = AutoModelForTokenClassification.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased")

# สร้าง NER pipeline
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/905k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/423M [00:00<?, ?B/s]

Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


In [6]:
import re
from pythainlp.util import normalize
from pythainlp.tokenize import word_tokenize
from fuzzywuzzy import fuzz

# ฐานข้อมูลสินค้า (DB ตัวอย่าง)
menu_db = ["น้ำแตงโม", "น้ำลำไย", "ชาเย็น", "กาแฟเย็น"]

def generate_bag_of_words(words, n=3):
    # สร้าง Bag of Words โดยการรวมคำต่อเนื่อง (1 ถึง n คำ)
    bag = []
    for size in range(1, n + 1):  # เริ่มตั้งแต่ 1-gram ถึง n-gram
        for i in range(len(words) - size + 1):
            bag.append(''.join(words[i:i + size]).strip())
    return bag

def find_best_match(word, menu_db, threshold=80):
    best_match = None
    highest_score = 0
    
    for menu in menu_db:
        score = fuzz.partial_ratio(normalize(word), normalize(menu))
        if score > highest_score and score >= threshold:
            best_match = menu
            highest_score = score
    
    return best_match

def extract_orders_with_bagword(text):
    orders = []
    used_indices = set()  # เก็บคำที่ใช้ไปแล้วเพื่อลดการตรวจจับซ้ำ
    detected_menus = set()  # เก็บเมนูที่ตรวจจับได้แล้วเพื่อป้องกันซ้ำ

    # Normalize ข้อความและตัดคำ
    text = normalize(text.strip())
    text = re.sub(r'\s+', ' ', text)  # ลบช่องว่างเกิน
    words = [w for w in word_tokenize(text) if w.strip()]  # กำจัดช่องว่างที่ไม่จำเป็น
    print("คำที่ตัดได้จากข้อความ:", words)

    # สร้าง Bag of Words
    bag_of_words = generate_bag_of_words(words)
    print("Bag of Words ที่สร้างได้:", bag_of_words)

    # ตรวจหาเมนูและจำนวนโดยใช้ Bag of Words
    for i, word in enumerate(bag_of_words):
        if i in used_indices:  # ถ้าคำนี้ถูกใช้แล้วให้ข้ามไป
            continue

        for menu in menu_db:
            # ตรวจจับคำที่เหมือนในระดับ 90% ขึ้นไป หรือใช้การเทียบ fuzzy
            if fuzz.ratio(normalize(menu), normalize(word)) >= 90 or find_best_match(word, menu_db):
                # ถ้าพบว่าเมนูนี้เคยเพิ่มไปแล้ว ให้ข้าม
                best_match = find_best_match(word, menu_db)
                if best_match and best_match in detected_menus:
                    continue

                # ตรวจสอบจำนวนที่ตามมา (ถ้ามี)
                quantity = 1
                if i + 1 < len(words) and words[i + 1].isdigit():
                    quantity = int(words[i + 1])
                    used_indices.add(i + 1)  # ข้ามจำนวนที่ใช้แล้ว
                
                orders.append({"menu": best_match if best_match else menu, "quantity": quantity})
                detected_menus.add(best_match if best_match else menu)  # เพิ่มเมนูนี้ในเซตเพื่อป้องกันการเพิ่มซ้ำ
                
                # ข้ามคำใน bag ที่เกี่ยวข้องทั้งหมด
                used_indices.update(range(i, i + len(menu.split())))
                break

    return orders

# ทดสอบด้วยข้อความ
text = "น้ำลำไย 4 น้ำแตงโม 1 กาแฟ 1 "
orders = extract_orders_with_bagword(text)
print("คำสั่งซื้อ:", orders)


คำที่ตัดได้จากข้อความ: ['น้ำ', 'ลำไย', '4', 'น้ำ', 'แตงโม', '1', 'กาแฟ', '1']
Bag of Words ที่สร้างได้: ['น้ำ', 'ลำไย', '4', 'น้ำ', 'แตงโม', '1', 'กาแฟ', '1', 'น้ำลำไย', 'ลำไย4', '4น้ำ', 'น้ำแตงโม', 'แตงโม1', '1กาแฟ', 'กาแฟ1', 'น้ำลำไย4', 'ลำไย4น้ำ', '4น้ำแตงโม', 'น้ำแตงโม1', 'แตงโม1กาแฟ', '1กาแฟ1']
คำสั่งซื้อ: [{'menu': 'น้ำแตงโม', 'quantity': 1}, {'menu': 'น้ำลำไย', 'quantity': 4}, {'menu': 'กาแฟเย็น', 'quantity': 1}]
